<h1 style="color:red";> voice assistant</h1>

<h1 style="color:Violet";>importing libraries</h1>

In [42]:
import torch


In [44]:
import soundfile as sf

In [ ]:
import sounddevice as sd

In [46]:
from faster_whisper import WhisperModel

In [48]:
from silero_vad import *

In [ ]:
import numpy as np

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
import asyncio
import edge_tts
import nest_asyncio


<h1 style=color:"Green";> Voice Assistant</h1>

In [2]:
# Function to record audio
def record_audio(duration, samplerate=16000):
    print(f"Recording for {duration} seconds...")
    audio_data = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype=np.float32)
    sd.wait()  # Wait until recording is finished
    print("Recording finished.")
    return audio_data.flatten()  # Return 1D array

In [5]:

# Function to chunk audio data
def chunk_audio(audio_data, chunk_size):
    for i in range(0, len(audio_data), chunk_size):
        yield audio_data[i:i + chunk_size]

In [64]:
import sounddevice as sd
import numpy as np
import whisper

# Initialize Whisper model
model = whisper.load_model("base")

# Define the duration of the audio recording in seconds
duration = 10

# Define the sample rate
samplerate = 16000

# Define the chunk size for processing
chunk_size = 16000

# Define the VAD model and threshold
# For this example, we'll use a simple threshold-based VAD
def simple_vad(audio, threshold=0.5):
    # Calculate the energy of the audio signal
    energy = np.sum(np.abs(audio) ** 2)
    # If the energy is above the threshold, consider it speech
    if energy > threshold:
        return True
    else:
        return False

vad_model = simple_vad
threshold = 0.5

# Define a list to store the full text output
full_text_output = []

def record_audio(duration, samplerate=16000):
    print(f"Recording for {duration} seconds...")
    audio_data = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype=np.float32)
    sd.wait()  # Wait until recording is finished
    print("Recording finished.")
    return audio_data.flatten()

def chunk_audio(audio_data, chunk_size):
    for i in range(0, len(audio_data), chunk_size):
        yield audio_data[i:i + chunk_size]

def get_speech_ts(audio, vad_model, threshold=0.5):
    # Implement your VAD logic here to detect speech segments
    # Return a list of dictionaries with 'start' and 'end' timestamps
    speech_segments = []
    if vad_model(audio, threshold):
        speech_segments.append({'start': 0, 'end': len(audio) / samplerate})
    return speech_segments

# Record audio
audio_data = record_audio(duration)

# Iterate over audio chunks and process each one
for chunk in chunk_audio(audio_data, chunk_size):
    # Detect speech segments within the chunk
    speech_segments = get_speech_ts(chunk, vad_model, threshold=0.5)

    if speech_segments is not None and len(speech_segments) > 0:
        # Process each detected speech segment
        for seg in speech_segments:
            start_sample = int(seg['start'] * samplerate)
            end_sample = int(seg['end'] * samplerate)
            speech_audio = chunk[start_sample:end_sample]
            
            # Convert speech audio to text using Whisper
            result = model.transcribe(speech_audio)
            text_output = result["text"]
            full_text_output.append(text_output)

# Join all the transcribed chunks into final text
final_transcription = " ".join(full_text_output)

print(f"Recorded  Text: {final_transcription}")

C:\Users\nikil\Downloads\ml projects\prj7\voice_assistant\Lib\site-packages\whisper\__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp

Recording for 10 seconds...
Recording finished.


C:\Users\nikil\Downloads\ml projects\prj7\voice_assistant\Lib\site-packages\whisper\transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Recorded  Text:  Thank you.  Thank you.  Thank you. Thank you.  Thank you.  Thank you.  Thank you.  Thank you.  Thank you.  Thank you.


<h1 style = "color:Green";>LLM  working </h1>

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import asyncio
import edge_tts
import nest_asyncio
import os

# Apply nest_asyncio to handle asyncio in Jupyter notebooks
nest_asyncio.apply()

async def text_to_speech(text, output_file, voice="en-US-JennyNeural", rate="+0%", pitch="+0Hz"):
    # Create a TTS engine with tunable parameters
    communicate = edge_tts.Communicate(text, voice=voice, rate=rate, pitch=pitch)
    await communicate.save(output_file)
    print(f"Audio saved to {output_file}")

# Define or set the input text
text_output = "Tell me the basics of Machine learning?"

# Load pre-trained GPT-2 model
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Encode the input text
inputs = tokenizer.encode(text_output, return_tensors="pt")

# Set pad_token_id to eos_token_id for open-end generation
pad_token_id = tokenizer.eos_token_id

# Generate output, limiting the response to 50 tokens
output_tokens = model.generate(
    inputs,
    max_length=50,
    num_return_sequences=1,
    pad_token_id=pad_token_id
)

# Decode the response
output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print(f"LLM Response: {output_text}")

# Convert the LLM response text to speech with custom pitch and rate
output_file = "output_audio.wav"
asyncio.run(text_to_speech(output_text, output_file, voice="en-US-JennyNeural", rate="-10%", pitch="+2Hz"))

# Check if the audio file was created and is not empty
if os.path.exists(output_file) and os.path.getsize(output_file) > 0:
    print("Audio file created successfully. Check your media player.")
else:
    print("Failed to create audio file or the file is empty.")


LLM Response: Tell me the basics of Machine learning?

Machine learning is a new field of research that has been around for a while. It is a new field of research that has been around for a while. It is a new field of research that has
Audio saved to output_audio.wav
Audio file created successfully. Check your media player.
